In [1]:
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import display, HTML
InteractiveShell.ast_node_interactivity = "all"
%config InlineBackend.figure_format='retina'

import warnings
warnings.filterwarnings('ignore')

In [6]:
import tmdbsimple as tmdb
tmdb.API_KEY = '08d3df3441c114a6cdb87682cb5b8013'

from editdistance import eval as editdistance

from surprise import Dataset
from surprise import accuracy
import numpy as np
import time
import pandas as pd
from pathlib import Path
from surprise.model_selection import train_test_split
data = Dataset.load_builtin('ml-1m')
trainset, testset = train_test_split(data, test_size=.25)
from tqdm import tqdm,tqdm_notebook
from joblib import Parallel, delayed

from hwer.utils import clean_text

In [3]:



path = Path(data.ratings_file)
ml_1m_dir = path.resolve().parents[1]
files = list(ml_1m_dir.glob('**/*.dat'))

users = [f for f in files if "users.dat" in str(f)][0]
movies = [f for f in files if "movies.dat" in str(f)][0]
ratings = [f for f in files if "ratings.dat" in str(f)][0]

users = pd.read_csv(str(users),sep="::", header=None, names=["user_id", "gender", "age", "occupation", "zip"], engine='python')
movies = pd.read_csv(str(movies),sep="::", header=None, names=["movie_id", "title", "genres"], engine='python')
ratings = pd.read_csv(str(ratings),sep="::", header=None, names=["user_id", "movie_id", "rating", "timestamp"], engine='python')

movies['genres'] = movies['genres'].apply(lambda x: x.lower().split('|'))
users['user_id'] = users['user_id'].astype(str)
movies['movie_id'] = movies['movie_id'].astype(str)
ratings['movie_id'] = ratings['movie_id'].astype(str)
ratings['user_id'] = ratings['user_id'].astype(str)

print(users.shape, movies.shape, ratings.shape)

user_item_affinities = list(map(lambda x: tuple([x[0], x[1], x[2]]), data.raw_ratings))


(6040, 5) (3883, 3) (1000209, 4)


In [4]:
movies.head(3)
movies['year'] = movies.title.apply(lambda x: x[-5:-1])
movies.title = movies.title.apply(lambda x: x[:-7])
movies.head(3)


,movie_id,title,genres
0,1,Toy Story (1995),"[animation, children's, comedy]"
1,2,Jumanji (1995),"[adventure, children's, fantasy]"
2,3,Grumpier Old Men (1995),"[comedy, romance]"


,movie_id,title,genres,year
0,1,Toy Story,"[animation, children's, comedy]",1995
1,2,Jumanji,"[adventure, children's, fantasy]",1995
2,3,Grumpier Old Men,"[comedy, romance]",1995


In [5]:
np.sum(movies.title.isna())
np.sum(movies.year.isna())
np.sum(movies.title.apply(len)==0)
np.sum(movies.year.apply(len)==0)

0

0

0

0

In [202]:
users.to_csv("users.csv", sep="\t", index=False)
ratings.to_csv("ratings.csv", sep="\t", index=False)

## Enhancing item features with TMDB

In [14]:
import re

def clean_text(text):
    EMPTY = ' '
    if text is None:
        return EMPTY
    
    text = text.replace("\n", " ").replace("(", " ").replace(")", " ").replace("\r", " ").replace("\t", " ").lower()
    text = re.sub('<pre><code>.*?</code></pre>', EMPTY, text)
    text = re.sub('<code>.*?</code>', EMPTY, text)

    def replace_link(match):
        return EMPTY if re.match('[a-z]+://', match.group(1)) else match.group(1)

    text = re.sub('<a[^>]+>(.*)</a>', replace_link, text)
    text = re.sub('<.*?>', EMPTY, text)
    return text

In [23]:
def get_movie_details_from_tmdb(movie_id, title, year):
    year = int(year)
    
    search = tmdb.Search()
    response = search.movie(query=title)
    results = response['results']
    
    if len(results) == 0:
        tn = title.split(",")[0]
        search = tmdb.Search()
        response = search.movie(query=tn)
        results = response['results']
        
    if len(results) == 0:
        tn = title.split(":")[0]
        search = tmdb.Search()
        response = search.movie(query=tn)
        results = response['results']
        
    if len(results) == 0:
        tn = title.split("(")[0]
        search = tmdb.Search()
        response = search.movie(query=tn)
        results = response['results']
    
    
    
    from functools import cmp_to_key
    def cmp(m1, m2):
        edst_1 = editdistance(title.lower(), m1['title'].lower())
        edst_2 = editdistance(title.lower(), m2['title'].lower())
        if 'release_date' not in m1:
            return 1
        if 'release_date' not in m2:
            return -1
        year_diff_1 = np.abs(pd.to_datetime(m1['release_date']).year - year)
        year_diff_2 = np.abs(pd.to_datetime(m2['release_date']).year - year)
        score_1 = 0.3 * edst_1 + year_diff_1
        score_2 = 0.3 * edst_2 + year_diff_2
        return -1 if score_1 <= score_2 else 1
    results = list(sorted(results, key=cmp_to_key(cmp)))
    if len(results) > 0:
        movie = tmdb.Movies(results[0]['id'])
        keywords = [k['name'] for k in movie.keywords()['keywords']]
        info = movie.info()
        original_language = info['original_language']
        overview = clean_text(info['overview'])
        runtime = info['runtime']
        tagline = clean_text(info['tagline'])
        original_title = info['original_title']
        title = info['title']
        release_date = info['release_date']
        return {"movie_id":movie_id,"title":title, "keywords":keywords, "original_language":original_language, 
                "overview":overview, "runtime":runtime, "tagline":tagline, 
                'original_title':original_title, "release_date":release_date,
               "success":True}
    else:
        return {"movie_id":movie_id,"title":title, "keywords":[], "original_language":'', 
                "overview":'', "runtime":-1, "tagline":'', 
                'original_title':'',"release_date":str(year), 
                "success":False}
    


In [24]:
get_movie_details_from_tmdb(movie_id=100,title="Toy Story", year=1995)

{'movie_id': 100,
 'title': 'Toy Story',
 'keywords': ['martial arts',
  'jealousy',
  'toy',
  'boy',
  'friendship',
  'bullying',
  'elementary school',
  'friends',
  'rivalry',
  'rescue',
  'mission',
  'walkie talkie',
  'boy next door',
  'new toy',
  'neighborhood',
  'toy comes to life',
  'resourcefulness'],
 'original_language': 'en',
 'overview': "led by woody, andy's toys live happily in his room until andy's birthday brings buzz lightyear onto the scene. afraid of losing his place in andy's heart, woody plots against buzz. but when circumstances separate buzz and woody from their owner, the duo eventually learns to put aside their differences.",
 'runtime': 81,
 'tagline': '',
 'original_title': 'Toy Story',
 'release_date': '1995-10-30',
 'success': True}

In [108]:
movies.head(3).apply(lambda m:get_movie_details_from_tmdb(m['title'],m['year'])['overview'], axis=1)

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
dtype: object

In [ ]:
tmdb_data = {}


In [29]:

titles_years = list(zip(movies['movie_id'], movies['title'],movies['year']))
# overviews = Parallel(n_jobs=8)(delayed(get_movie_details_from_tmdb)(title,year) for title,year in tqdm_notebook(titles_years))


for movie_id,title,year in tqdm_notebook(titles_years):
    if movie_id in tmdb_data:
        continue
    movie_detail = get_movie_details_from_tmdb(movie_id=movie_id, title=title, year=year)
    tmdb_data[movie_id] = movie_detail




In [34]:
unsuccessful =[k for k,v in tmdb_data.items() if not v['success']]
len(unsuccessful)


30

In [76]:
movies[movies.movie_id.isin(unsuccessful)].head(30).tail(5)

,movie_id,title,genres,year
3508,3577,Two Moon Juction,[drama],1988
3693,3762,Daughter of Dr. Jeckyll,[horror],1957
3730,3799,Pok�mon the Movie 2000,"[animation, children's]",2000
3780,3850,Whatever Happened to Aunt Alice?,"[crime, thriller]",1969
3784,3854,Aim�e & Jaguar,"[drama, romance]",1999


In [75]:
movie_id = "3854"
movie_detail = get_movie_details_from_tmdb(movie_id=movie_id, title="Jaguar", year=1999)
movie_detail
if movie_detail["success"]:
    tmdb_data[movie_id] = movie_detail
else:
    print("Fail")


{'movie_id': '3854',
 'title': 'Aimee & Jaguar',
 'keywords': ['berlin germany',
  'world war ii',
  'jew',
  'forbidden love',
  'lesbian relationship',
  'homosexuality',
  'nazi germany',
  'lesbian interest'],
 'original_language': 'de',
 'overview': 'in 1943, while the allies are bombing berlin and the gestapo is purging the capital of jews, a dangerous love affair blossoms between two women - one a jewish member of the underground, the other an exemplar of nazi motherhood.',
 'runtime': 125,
 'tagline': 'bound by desire. torn apart by war.',
 'original_title': 'Aimee & Jaguar',
 'release_date': '1999-02-10',
 'success': True}

In [77]:
unsuccessful =[k for k,v in tmdb_data.items() if not v['success']]
len(unsuccessful)


12

In [83]:
tmdb_df = pd.DataFrame.from_records(list(tmdb_data.values()))
tmdb_df.drop(columns=["success"], inplace=True)
tmdb_df.shape
assert tmdb_df.shape[0] == len(list(tmdb_data.values()))

tmdb_df.to_csv("tmdb_data.csv", sep="\t", index=False)


(3883, 9)

In [85]:
movies.shape
tmdb_df.rename(columns={"title":"tmdb_title"}, inplace=True)
movies = movies.merge(tmdb_df, on="movie_id")
movies.shape


(3883, 4)

(3883, 12)

In [87]:
movies.to_csv("movies.csv", sep="\t", index=False)

In [94]:

user_movie_counts = ratings.groupby(["user_id"])[["movie_id"]].count()
user_movie_counts.head()

user_movie_counts.sort_values(by=["movie_id"]).head(100)

,movie_id
user_id,
1,53
10,401
100,76
1000,84
1001,377


,movie_id
user_id,
5145,20
2381,20
4178,20
4192,20
421,20
...,...
5899,21
5052,21
653,21


In [5]:
movies = pd.read_csv("movies.csv", sep="\t", engine="python")
users = pd.read_csv("users.csv", sep="\t")
ratings = pd.read_csv("ratings.csv", sep="\t")


